# Procesamiento del dataset

El objetivo principal de este cuaderno es transformar el dataset `components_01.csv`, obtenido a partir del proceso de creación y limpieza de datos realizado tras el scraping de diferentes sitios web en el cuaderno `create_and_clean_dataset.ipynb`. El resultado será un nuevo dataset que contenga únicamente el target de nuestro futuro modelo en valor numérico y las rutas locales de las imágenes de los componentes.

En esta fase, se automatizará la descarga de imágenes a partir de sus URLs y se almacenarán sus rutas locales en el dataset procesado.

Además, se mapeará la columna `label` del dataset a valores enteros, dejándolo preparado para el posterior entrenamiento del modelo.

### 👨‍💻 Autores del proyecto

* [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)
* [Alvaro López Guerrero](https://github.com/Alvalogue72)
* [Andrei Munteanu Popa](https://github.com/andu8705)

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing - `Curso 2025/2026`

## Importaciones de paquetes

In [ ]:
import pandas as pd
import requests
import os
import time

## Carga del dataset `dataset_01.ipynb`
Cargamos el dataset previamente generado en el cuaderno `create_and_clean_dataset.ipynb`. Este dataset contiene la información de los componentes, incluyendo las URLs de las imágenes que vamos a descargar.

In [27]:
df = pd.read_csv('../data/processed/components_01.csv')
df

,image_url,label
0,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard
1,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard
2,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard
3,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard
4,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard
...,...,...
9977,https://cdna.pcpartpicker.com/static/forever/i...,hard_drive
9978,https://cdna.pcpartpicker.com/static/forever/i...,hard_drive
9979,https://cdna.pcpartpicker.com/static/forever/i...,hard_drive
9980,https://m.media-amazon.com/images/I/31s-wKnYDv...,hard_drive


## Descarga de imágenes y generación de rutas locales
Definimos una función para descargar imágenes a partir de una URL y guardarlas localmente con un nombre específico. Esto nos permitirá asociar cada imagen con su ruta local en el dataset.

Creamos la carpeta de imágenes si no existe y recorremos el dataset descargando cada imagen. Guardamos la ruta local de cada imagen para su posterior uso en el modelo.

In [ ]:
def download_img(url, filename):
    """Función para descargar una imagen desde una URL y guardarla localmente."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [ ]:
img_dir = '../data/images'
os.makedirs(img_dir, exist_ok=True)

local_paths = []
for idx, row in df.iterrows():
    url = row['image_url']
    label = row['label']
    ext = os.path.splitext(url)[-1].split('?')[0]
    filename = f"{label}_{idx}{ext if ext else '.jpg'}"
    filepath = os.path.join(img_dir, filename)
    local_path = download_img(url, filepath)
    local_paths.append(local_path)
    time.sleep(0.1)

Error downloading https://thumb.pccomponentes.com/w-530-530/articles/39/398851/1888-asrock-b550m-phantom-gaming-4.jpg: 404 Client Error: Not Found for url: https://thumb.pccomponentes.com/w-530-530/articles/39/398851/1888-asrock-b550m-phantom-gaming-4.jpg
Error downloading https://thumb.pccomponentes.com/w-530-530/articles/1066/10661014/productimage6-93884767-f415-4d6a-a72a-b7d5adc165ce.jpg: 404 Client Error: Not Found for url: https://thumb.pccomponentes.com/w-530-530/articles/1066/10661014/productimage6-93884767-f415-4d6a-a72a-b7d5adc165ce.jpg
Error downloading https://thumb.pccomponentes.com/w-530-530/articles/1066/10661027/productimage6-da22a662-c9cd-4d19-8092-980015d17eb9.jpg: 404 Client Error: Not Found for url: https://thumb.pccomponentes.com/w-530-530/articles/1066/10661027/productimage6-da22a662-c9cd-4d19-8092-980015d17eb9.jpg
Error downloading https://thumb.pccomponentes.com/w-530-530/articles/1038/10384667/productimage6-5d642cbb-79c0-4294-b8d0-57a783e28fbe.jpg: 404 Client Er

,image_url,label,local_image_path
0,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard,../data/images\motherboard_0.jpg
1,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard,../data/images\motherboard_1.jpg
2,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard,../data/images\motherboard_2.jpg
3,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard,../data/images\motherboard_3.jpg
4,https://thumb.pccomponentes.com/w-530-530/arti...,motherboard,../data/images\motherboard_4.jpg


## Guardado del dataset (checkpoint)
Se añade al DataFrame una columna con la ruta local de cada imagen y se guarda el nuevo dataset procesado. Este paso actúa como un *checkpoint*, ya que el dataset se utilizará en futuros modelos y servirá además como respaldo.

In [ ]:
df['local_image_path'] = local_paths
df.to_csv('../data/processed/components_02.csv', index=False)

## Preparación de los datos

Actualmente, el dataset `components_02.csv` contiene **9982 registros**. Disponemos del **tipo de componente** (`label`), que será el *target* de nuestro modelo inicial, así como la URL de la imagen (`image_url`) y la ruta local (`local_image_path`), aunque mantiene fallos por temas de barras y valores nulos, que ahora arreglaremos.

| #  | Columnas           | Non-Null Count | Tipos |
|----|-----------------|----------------|-------|
| 0  | image_url        | 9982           | str   |
| 1  | label            | 9982           | str   |
| 2  | local_image_path | 9971           | str   |

In [29]:
df_02 = pd.read_csv('../data/processed/components_02.csv')
df_02.info()

<class 'pandas.DataFrame'>
RangeIndex: 9982 entries, 0 to 9981
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   image_url         9982 non-null   str  
 1   label             9982 non-null   str  
 2   local_image_path  9971 non-null   str  
dtypes: str(3)
memory usage: 234.1 KB


### Valores nulos
Identificamos y eliminamos los registros que contienen valores nulos. Estos valores proceden de fallos de descarga durante el proceso de obtención de las imágenes, debido a la inexistencia de las mismas. Dado que no disponemos de imágenes que identifiquen nuestros componentes, prescindimos de esos registros.

In [30]:
df_02.isnull().sum()

image_url            0
label                0
local_image_path    11
dtype: int64

In [31]:
df_02 = df_02.dropna().reset_index(drop=True)
df_02.isnull().sum()

image_url           0
label               0
local_image_path    0
dtype: int64

### Normalización de rutas locales de imágenes
Ajustamos el formato de las rutas locales de las imágenes y arreglamos los problemas con las barras, para que sean rutas relativas sin errores. Esto facilita el acceso a las imágenes.

In [32]:
df_02["local_image_path"]

0         ../data/images\motherboard_0.jpg
1         ../data/images\motherboard_1.jpg
2         ../data/images\motherboard_2.jpg
3         ../data/images\motherboard_3.jpg
4         ../data/images\motherboard_4.jpg
                       ...                
9966    ../data/images\hard_drive_9977.jpg
9967    ../data/images\hard_drive_9978.jpg
9968    ../data/images\hard_drive_9979.jpg
9969    ../data/images\hard_drive_9980.jpg
9970    ../data/images\hard_drive_9981.jpg
Name: local_image_path, Length: 9971, dtype: str

In [33]:
df_02["local_image_path"] = df_02["local_image_path"].str.replace(r'\\', '/', regex=True)
df_02["local_image_path"]

0         ../data/images/motherboard_0.jpg
1         ../data/images/motherboard_1.jpg
2         ../data/images/motherboard_2.jpg
3         ../data/images/motherboard_3.jpg
4         ../data/images/motherboard_4.jpg
                       ...                
9966    ../data/images/hard_drive_9977.jpg
9967    ../data/images/hard_drive_9978.jpg
9968    ../data/images/hard_drive_9979.jpg
9969    ../data/images/hard_drive_9980.jpg
9970    ../data/images/hard_drive_9981.jpg
Name: local_image_path, Length: 9971, dtype: str

### Eliminar la columna `image_url`
Eliminamos la columna de la URL original de la imagen, ya que ahora solo necesitamos la ruta local. Comprobamos la estructura final del DataFrame para asegurarnos de que está listo.

In [34]:
df_02 = df_02.drop(columns=["image_url"])
df_02.info()

<class 'pandas.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   label             9971 non-null   str  
 1   local_image_path  9971 non-null   str  
dtypes: str(2)
memory usage: 155.9 KB


## Transformar la etiqueta a números

Para que sea más eficiente, transforma la columna `label` a números.

| Número | Componente       |
|--------|-----------------|
| 0      | motherboard     |
| 1      | gpu             |
| 2      | cpu             |
| 3      | hard_drive      |
| 4      | ram             |
| 5      | pc_case         |
| 6      | power_supply    |
| 7      | liquid_cooling  |
| 8      | case_fan        |
| 9      | cpu_fan         |
| 10     | sound_card      |

In [35]:
df_02["label_num"], label_mapping = pd.factorize(df_02["label"])
label_map = {i: name for i, name in enumerate(label_mapping)}

print(label_map)


{0: 'motherboard', 1: 'gpu', 2: 'cpu', 3: 'hard_drive', 4: 'ram', 5: 'pc_case', 6: 'power_supply', 7: 'liquid_cooling', 8: 'case_fan', 9: 'cpu_fan', 10: 'sound_card'}


In [36]:
df_02.info()

<class 'pandas.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   label             9971 non-null   str  
 1   local_image_path  9971 non-null   str  
 2   label_num         9971 non-null   int64
dtypes: int64(1), str(2)
memory usage: 233.8 KB


Ahora se borrará el label original y se cambiará el nombre al numérico.

In [37]:
df_02 = df_02.drop(columns=["label"])
df_02 = df_02.rename(columns={"label_num": "label"})

In [38]:
df_02.info()

<class 'pandas.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   local_image_path  9971 non-null   str  
 1   label             9971 non-null   int64
dtypes: int64(1), str(1)
memory usage: 155.9 KB


## Guardado del dataset (checkpoint)
Guardamos el DataFrame final en un archivo CSV. Este archivo contiene únicamente las columnas necesarias: la etiqueta del componente y la ruta local de la imagen. Volvemos a hacer un *checkpoint*.

In [39]:
df_02.to_csv('../data/processed/components_03.csv', index=False)

## Conclusiones

En esta cuaderno hemos procesado el dataset, a falta del redimensionado y vectorizado de imágenes.